In [6]:
import numpy as np
import pandas as pd

In [7]:
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
#Feature Engineering

In [9]:
movies = pd.read_csv('/Users/kamal/Documents/Datasets/Movie Dataset/movies.csv')
movies.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [10]:
movies.shape

(4803, 24)

In [53]:
features = ['genres','keywords','cast','tagline','director','popularity']
print(features)
movies[features]

['genres', 'keywords', 'cast', 'tagline', 'director', 'popularity']


,genres,keywords,cast,tagline,director,popularity
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,Enter the World of Pandora.,James Cameron,150.437577
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,"At the end of the world, the adventure begins.",Gore Verbinski,139.082615
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,A Plan No One Escapes,Sam Mendes,107.376788
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,The Legend Ends,Christopher Nolan,112.312950
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"Lost in our world, found in another.",Andrew Stanton,43.926995
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"He didn't come looking for trouble, but troubl...",Robert Rodriguez,14.269792
4799,Comedy Romance,,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,A newlywed couple's honeymoon is upended by th...,Edward Burns,0.642552
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,,Scott Smith,1.444476
4801,,,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,A New Yorker in Shanghai,Daniel Hsia,0.857008


In [54]:
for row in features:
    movies[row] = movies[row].fillna('')

In [91]:
new_data = movies['genres']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['cast']+' '+movies['director']

In [92]:
print(new_data)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [93]:
#Text to Feature Vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(new_data)

In [94]:
feature_vectors.shape

(4803, 17318)

In [95]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [96]:
#Cosine Similarity
sim = cosine_similarity(feature_vectors)
print(sim)
print(sim.shape)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]
(4803, 4803)


In [97]:
#Test Similarity Scores
#Input Movie
name = input('Enter Movie Name: ')

Enter Movie Name: a space odyssey


In [98]:
#Close Match
names = movies['title'].tolist()
close_match = difflib.get_close_matches(name, names)
print(close_match)
print('Closest Match : ',close_match[0])

movie_by_index = int(movies[movies.title == close_match[0]]['index'])
print('Movie Index: '+str(movie_by_index))

['2001: A Space Odyssey']
Closest Match :  2001: A Space Odyssey
Movie Index: 2966


In [99]:
similarity_score = list(enumerate(sim[movie_by_index]))
print(len(similarity_score))

4803


In [100]:
#Similar movies
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

i=0
print('Movies similar to: ')
for movie in sorted_similar_movies:
    movie_title = movies[movies.index == movie[0]]['title'].values[0]
    if (i<31):
        print(i,' : '+ movie_title)
        i+=1    

    

Movies similar to: 
0  : 2001: A Space Odyssey
1  : The Matrix Revolutions
2  : I, Robot
3  : Blade Runner
4  : The Matrix Reloaded
5  : Terminator 3: Rise of the Machines
6  : The Matrix
7  : Eyes Wide Shut
8  : Star Trek: The Motion Picture
9  : Jupiter Ascending
10  : 9
11  : Automata
12  : Eagle Eye
13  : Terminator Salvation
14  : Transcendence
15  : Ex Machina
16  : Barry Lyndon
17  : Moon
18  : The Helix... Loaded
19  : Red Riding: In the Year of Our Lord 1974
20  : Lolita
21  : Apollo 18
22  : Terminator Genisys
23  : The Shining
24  : The X Files
25  : Virtuosity
26  : The Other Side of Heaven
27  : The Time Machine
28  : Interstellar
29  : TRON: Legacy
30  : Contact
